In [1]:
spark

NameError: name 'spark' is not defined

# 1 먼저 코퍼스 파일을 읽어 온다. 

hdfs 없으면 로컬에서 읽어오고, 파일이 있다면 hdfs에서 읽어온다.  
앞에 프로토콜을 지정해 줄수도 있다. 

In [11]:
df = spark.read.text('file:///notebook/*.txt')

NameError: name 'spark' is not defined

## 읽은 파일은 Dataframe 으로 가져온다. 잘 가져왔는지 확인한다.


In [3]:
df.columns

NameError: name 'df' is not defined

In [4]:
df.count()

NameError: name 'df' is not defined

In [5]:
df.take(1)

NameError: name 'df' is not defined

In [6]:
df.show()

NameError: name 'df' is not defined

# 이미 만들어진 Dataframe 객체에서 무엇을 할 수 있을까?




In [7]:
# 다른 형태로 쪼개서 
# 바로 RDD 들어오는게 아니라 Row 라는 객체가 들어 있다. 
# 여기서 여러가지 작업을 해서 최종적으로 다시 테이블로 변경해도 된다?
# 바로 상위 df API 에서 작업하는 것과 무엇이 차이나는가?
# 더 빨라지는가?



In [8]:
import re 

def extract_id_and_doc(text: str, pattern: str = None) -> tuple:
    """Return a tuple divided by a given pattern
    This function search for a string having a given pattern and extract the first,
    then return the rest of string with the extracted one. 

    Args:
        text: Target to extract.
        pattern: Regex pattern. Default is 
        
    Returns:
        A pair of tuple like (extracted string, rest of string).
        Extracted string is empty if no patten matches, as the rest if the same as input text
 """
    
    DEFAULT_PATTERN = r'([\@\!\$\%\^\&\#\w\d]+)'
    REG_PATTERN = DEFAULT_PATTERN if not pattern else pattern 
    DEFAULT_NULL_DATA = 'NODATA'
    
    def extract_id(text):
        try:
            # Include all char in coca files
            # Should add more if you find a new pattern 
            match = re.search(REG_PATTERN, text)
            if match:
                id = match.group(1)
            else:
                id = DEFAULT_NULL_DATA
        except:
            id = DEFAULT_NULL_DATA
        return id 

    def extract_doc_with_id(doc_id, text):

        if  doc_id ==DEFAULT_NULL_DATA:
            return DEFAULT_NULL_DATA
        else:
            # Strip id
            doc = text.strip(doc_id)
            return doc
    
    # Strip original text
    striped_input = text.strip('\n').strip('\t')
    doc_id = extract_id(striped_input)
    doc = extract_doc_with_id(doc_id, striped_input)
    # Strip document
    striped_doc = doc.strip()
    
    return (doc_id, striped_doc)

In [9]:
id_text = ex.map(lambda line: extract_id_and_doc(line)) # [ (id, text), ...  ]

NameError: name 'ex' is not defined

In [10]:
id_text.take(2)

NameError: name 'id_text' is not defined

# 문장으로 어떻게 나누는가?

   

In [146]:
def sent(text: str, include: bool = False) -> tuple:
    if include:
        return (text.split('.'),text)
    else:
        return (text.split('.'),)
    
def pair_sent(pair: tuple) -> list:
    id = pair[0]
    text = pair[1]
    
    result = []
    for sen in sent(text)[0]:
        result.append((id, sen))
        
    return result

In [147]:
id_sents = id_text.flatMap(lambda pair: pair_sent(pair))

In [148]:
id_sents.take(3)

[('@@8013891',
  '<h> Dealogic League Table Round-up April 1 <p> A total of $952 million has been raised in the Asian equity capital markets through 12 deals this week , down from the $6'),
 ('@@8013891', '8 billion via 31 deals completed in the prior week '),
 ('@@8013891',
  " Of the 12 deals priced this week , Shanghai Flyco Electrical Appliance 's $128 million IPO is the largest IPO to complete this week on April 1st , 2016 and marks the tenth biggest new listing* in the region so far this year ")]

In [149]:
id_sents.count()

31536

# 문장을 어떻게 토큰으로 나누는가?

In [162]:
def tokenize(text: str, include: bool = False) -> tuple:
    if include:
        return (text.split(),text)
    else:
        return (text.split(),)
    
def pair_tokenize(pair: tuple, include = False) -> list:
    id = pair[0]
    text = pair[1]
    
    result = []
    for tok in tokenize(text)[0]:
        if include:
            result.append((id, tok, text))
        else:
            result.append((id, tok))
    return result

In [167]:
id_toks = id_sents.flatMap(lambda pair: pair_tokenize(pair, include=False))

In [168]:
id_toks_sent = id_sents.flatMap(lambda pair: pair_tokenize(pair, include=True))

In [169]:
id_toks.take(5)

[('@@8013891', '<h>'),
 ('@@8013891', 'Dealogic'),
 ('@@8013891', 'League'),
 ('@@8013891', 'Table'),
 ('@@8013891', 'Round-up')]

In [170]:
id_toks_sent.take(2)

[('@@8013891',
  '<h>',
  '<h> Dealogic League Table Round-up April 1 <p> A total of $952 million has been raised in the Asian equity capital markets through 12 deals this week , down from the $6'),
 ('@@8013891',
  'Dealogic',
  '<h> Dealogic League Table Round-up April 1 <p> A total of $952 million has been raised in the Asian equity capital markets through 12 deals this week , down from the $6')]

In [1]:
id_toks_sent

NameError: name 'id_toks_sent' is not defined